# 감성 평가

 형태소 분석을 통한 단어 중심 감성평가

## 형태소 분석
 Okt, mecab 

In [ ]:
# =========================
# 설치
# =========================
!pip -q install konlpy mecab-python3 emoji pandas


### mecab 한국어 설치
!pip install python-mecab-ko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 22.9 MB/s eta 0:00:00
[알림] Mecab 사용 불가 → Okt로 진행합니다.
사용 형태소 분석기: okt


,raw,norm,base,intensity,emoji_boost,total,label,intensity_detail,emoji_detail
0,레전드 공연... 무대미쳤다 😂😂 강추,레전드 공연... 무대미쳤다 😂😂 강추,2,0.000,0.989,2.989,positive,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😂': 2}
1,가격창렬에 음향구림.. 환불각 😡,가격창렬에 음향구림.. 환불각 😡,-1,0.000,-0.624,-1.624,negative,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😡': 1}
2,노잼이었는데 ㅠㅠ 배우 연기는 좋았다,노잼이었는데 ᅲᅲ 배우 연기는 좋았다,0,-0.769,0.000,-0.769,neutral,"{'k': 0, 'h': 0, 'tt': 2, 'ts': 0}",{}
3,대박 갓작!! 재관람 의사 100% 🤩🤩,대박 갓작! 재관람 의사 100% 🤩🤩,1,0.000,0.989,1.989,positive,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'🤩': 2}
4,좌석별로 시야가 너무 별로였음 😞,좌석별로 시야가 너무 별로였음 😞,-2,0.000,-0.485,-2.485,negative,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😞': 1}
5,그럭저럭이었지만 ㅎㅎ 분위기는 좋았음,그럭저럭이었지만 ᄒᄒ 분위기는 좋았음,1,0.549,0.000,1.549,positive,"{'k': 0, 'h': 2, 'tt': 0, 'ts': 0}",{}
6,콘서트 진행엉망.. 최악 😭😭,콘서트 진행엉망.. 최악 😭😭,-1,0.000,-1.099,-2.099,negative,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😭': 2}


In [1]:

import re, emoji, unicodedata
import numpy as np
import pandas as pd
from konlpy.tag import Okt

# =========================
# 형태소(Mecab → Okt 폴백)
# =========================
tokenizer_name="okt"; okt=Okt(); mecab=None
try:
    from konlpy.tag import Mecab
    mecab=Mecab(); tokenizer_name="mecab"
except:
    print("[알림] Mecab 사용 불가 → Okt로 진행합니다.")
print("사용 형태소 분석기:", tokenizer_name)

def morphs(text, stem=True):
    if tokenizer_name=="mecab":
        return [t for t in mecab.morphs(text) if len(t)>=2]
    else:
        return [t for t in okt.morphs(text, stem=stem) if len(t)>=2]

# =========================
# 정규화 + 웃음/울음 강도
# =========================
PATTERNS = {
    r"(ㅋ)\1{1,}": "ㅋㅋ",
    r"(ㅎ)\1{1,}": "ㅎㅎ",
    r"(ㅠ)\1{1,}": "ㅠㅠ",
    r"(ㅜ)\1{1,}": "ㅜㅜ",
    r"(!)\1{1,}": "!",
    r"([?])\1{1,}": "?",
}
def normalize_text(raw: str):
    s = unicodedata.normalize("NFKC", raw)
    for pat, rep in PATTERNS.items():
        s = re.sub(pat, rep, s)
    # 이모지 자체는 감성 점수에 쓰므로 여기선 보존(치환 X)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def count_intensity(raw: str):
    """ㅋ/ㅎ/ㅠ/ㅜ 반복 묶음 총 길이(강도) 합산"""
    return {
        "k": sum(len(m.group(0)) for m in re.finditer(r"(ㅋ)\1*", raw)),
        "h": sum(len(m.group(0)) for m in re.finditer(r"(ㅎ)\1*", raw)),
        "tt": sum(len(m.group(0)) for m in re.finditer(r"(ㅠ)\1*", raw)),
        "ts": sum(len(m.group(0)) for m in re.finditer(r"(ㅜ)\1*", raw)),
    }

# =========================
# 도메인 특화 사전(예시: 공연/콘서트/영화 리뷰)
# =========================
POS_LEX_BASE = set("좋다 최고 재밌다 대박 행복 웃기다 만족 설렌다 멋지다 감동 명작 몰입 꿀잼 히트".split())
NEG_LEX_BASE = set("별로 최악 싫다 지루하다 짜증 화남 분노 실망 아쉽다 우울 슬프다 노잼 망작 혹평 불친절 불만".split())

# 도메인 추가 단어(원하는 만큼 확장)
POS_DOMAIN = set("떡상 혜자 갓작 미쳤다 레전드 몰입감 캐스팅굿 연출굿 콘서트짱 무대미쳤다 갓연기 강추 재관람".split())
NEG_DOMAIN = set("창렬 병맛 똥망 개노잼 음향구림 좌석별로 발연기 환불각 개실망 민폐 가격창렬 진행엉망".split())

POS_LEX = POS_LEX_BASE | POS_DOMAIN
NEG_LEX = NEG_LEX_BASE | NEG_DOMAIN

def base_sentiment_score(text: str):
    toks = morphs(text, stem=True)
    score = 0
    for t in toks:
        if t in POS_LEX: score += 1
        if t in NEG_LEX: score -= 1
    return score

# =========================
# 이모지 감성 가중치(가벼운 사전)
# =========================
# 가중치는 예시값: 각자 설정에 맞춰 조절하세요.
EMOJI_WEIGHT = {
    "😂": +0.9, "😆": +0.8, "🤣": +1.0, "😊": +0.6, "😄": +0.7, "😍": +0.9, "🤩": +0.9, "😅": +0.5,
    "😭": -1.0, "😢": -0.8, "😡": -0.9, "🤬": -1.0, "😔": -0.7, "😞": -0.7, "🤢": -0.8, "😐": -0.1
}

def extract_emoji_counts(raw: str):
    """문자열 내 이모지별 빈도 dict"""
    counts = {}
    for ch in raw:
        if emoji.is_emoji(ch):
            counts[ch] = counts.get(ch, 0) + 1
    return counts

def emoji_score(raw: str, use_log=True):
    counts = extract_emoji_counts(raw)
    s = 0.0
    for e, c in counts.items():
        w = EMOJI_WEIGHT.get(e, 0.0)
        v = np.log1p(c) if use_log else c  # 반복 이모지 완만화
        s += w * v
    return s, counts

# =========================
# 최종 점수: 베이스 + 강도 + 이모지
# =========================
def combined_sentiment(raw: str,
                       w_k=0.5, w_h=0.5, w_tt=-0.7, w_ts=-0.7,
                       use_log_intensity=True, use_log_emoji=True,
                       pos_th=1.0, neg_th=-1.0):
    text = normalize_text(raw)

    base = base_sentiment_score(text)

    intens = count_intensity(raw)  # 원글 기준(정규화 전)으로 강도 포착
    if use_log_intensity:
        k = np.log1p(intens["k"]); h = np.log1p(intens["h"])
        tt = np.log1p(intens["tt"]); ts = np.log1p(intens["ts"])
    else:
        k,h,tt,ts = intens["k"], intens["h"], intens["tt"], intens["ts"]
    boost_intensity = w_k*k + w_h*h + w_tt*tt + w_ts*ts

    boost_emoji, emo_counts = emoji_score(raw, use_log=use_log_emoji)

    total = base + boost_intensity + boost_emoji
    label = "neutral"
    if total >= pos_th: label = "positive"
    if total <= neg_th: label = "negative"

    return {
        "raw": raw,
        "norm": text,
        "base": base,
        "intensity": round(float(boost_intensity), 3),
        "emoji_boost": round(float(boost_emoji), 3),
        "total": round(float(total), 3),
        "label": label,
        "intensity_detail": intens,
        "emoji_detail": emo_counts
    }

# =========================
# 추가 데모
# =========================
SENTS = [
    "레전드 공연... 무대미쳤다 😂😂 강추",
    "가격창렬에 음향구림.. 환불각 😡",
    "노잼이었는데 ㅠㅠ 배우 연기는 좋았다",
    "대박 갓작!! 재관람 의사 100% 🤩🤩",
    "좌석별로 시야가 너무 별로였음 😞",
    "그럭저럭이었지만 ㅎㅎ 분위기는 좋았음",
    "콘서트 진행엉망.. 최악 😭😭",
]

rows = [combined_sentiment(s) for s in SENTS]
df = pd.DataFrame(rows, columns=[
    "raw","norm","base","intensity","emoji_boost","total","label","intensity_detail","emoji_detail"
])
df


사용 형태소 분석기: mecab


,raw,norm,base,intensity,emoji_boost,total,label,intensity_detail,emoji_detail
0,레전드 공연... 무대미쳤다 😂😂 강추,레전드 공연... 무대미쳤다 😂😂 강추,2,0.000,0.989,2.989,positive,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😂': 2}
1,가격창렬에 음향구림.. 환불각 😡,가격창렬에 음향구림.. 환불각 😡,0,0.000,-0.624,-0.624,neutral,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😡': 1}
2,노잼이었는데 ㅠㅠ 배우 연기는 좋았다,노잼이었는데 ᅲᅲ 배우 연기는 좋았다,0,-0.769,0.000,-0.769,neutral,"{'k': 0, 'h': 0, 'tt': 2, 'ts': 0}",{}
3,대박 갓작!! 재관람 의사 100% 🤩🤩,대박 갓작! 재관람 의사 100% 🤩🤩,1,0.000,0.989,1.989,positive,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'🤩': 2}
4,좌석별로 시야가 너무 별로였음 😞,좌석별로 시야가 너무 별로였음 😞,-1,0.000,-0.485,-1.485,negative,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😞': 1}
5,그럭저럭이었지만 ㅎㅎ 분위기는 좋았음,그럭저럭이었지만 ᄒᄒ 분위기는 좋았음,0,0.549,0.000,0.549,neutral,"{'k': 0, 'h': 2, 'tt': 0, 'ts': 0}",{}
6,콘서트 진행엉망.. 최악 😭😭,콘서트 진행엉망.. 최악 😭😭,-1,0.000,-1.099,-2.099,negative,"{'k': 0, 'h': 0, 'tt': 0, 'ts': 0}",{'😭': 2}
